In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [2]:
import os

In [3]:
path = "D:\\Long Class Data\\DE\\Recent class\\Dataset\\log_search\\"

In [4]:
spark = SparkSession.builder.appName("log_search").getOrCreate()

In [5]:
largest_original_file = None
for i in range(1,15):
    if i <10:
        largest_file = 0
        filename = "2022060" + str(i)
        for data_file in os.listdir(path + filename):
            file_size = os.path.getsize(path+filename+'\\'+data_file)
            if int(file_size) > largest_file:
                largest_filename = path + filename + '\\' + data_file
                largest_file = file_size
            else:
                continue
    else:
        filename = "202206" + str(i)
        largest_file = 0
        for data_file in os.listdir(path + filename):
            file_size = os.path.getsize(path+filename+'\\'+data_file)
            if int(file_size) > largest_file:
                largest_filename = path + filename + '\\' + data_file
                largest_file = file_size
    if largest_original_file == None:
        largest_file = largest_filename.replace("'\'","\\")
        read_newfile = spark.read.parquet(largest_file)
        largest_original_file = read_newfile
        print(f'Done {i}')
    else:
        largest_file = largest_filename.replace("'\'","\\")
        read_newfile = spark.read.parquet(largest_file)
        largest_original_file = largest_original_file.union(read_newfile)
        print(f'Done {i}')          


Done 1
Done 2
Done 3
Done 4
Done 5
Done 6
Done 7
Done 8
Done 9
Done 10
Done 11
Done 12
Done 13
Done 14


In [6]:
ds = largest_original_file

In [7]:
ds= ds.select('datetime','user_id','keyword')

In [8]:
ds = ds.withColumn('datetime',col('datetime').cast('timestamp'))

In [9]:
ds_month6 = ds

In [10]:
ds_month6.groupBy('user_id').count().show()

+--------+-----+
| user_id|count|
+--------+-----+
|45538068|    3|
|41322920|    3|
| 2976590|    1|
| 1377091|    4|
|49349263|    5|
|49141594|    5|
|93203791|    1|
|41104363|    1|
|41892277|    5|
| 1676078|    2|
|06404339|    1|
| 7782530|    1|
|93542939|    3|
|96874946|    3|
| 0393163|    5|
|46583765|   13|
|42634326|   25|
|41202618|    1|
| 7608642|   15|
| 9744436|    1|
+--------+-----+
only showing top 20 rows



In [11]:
ds_month6.show()

+--------------------+--------+--------------------+
|            datetime| user_id|             keyword|
+--------------------+--------+--------------------+
|2022-06-01 18:59:...|    NULL|            trữ tình|
|2022-06-01 18:59:...|44887906|            trữ tình|
|2022-06-01 18:59:...| 2719170|              bolero|
|2022-06-01 15:00:...|    NULL|amy schumer: trực...|
|2022-06-01 19:00:...| 8830996|cậu mang à sĩ hanako|
|2022-06-01 19:00:...|    NULL|      Hoa trong bao |
|2022-06-01 19:00:...|41559909|liên minh công lý...|
|2022-06-01 19:00:...|92715770|                NULL|
|2022-06-01 19:00:...|49026196|    việt nam vs appa|
|2022-06-01 19:00:...|    NULL|chuyển sinh thành...|
|2022-06-01 19:00:...|41376437|nhất kiến khuynh tâm|
|2022-06-01 19:00:...| 1254139|                giác|
|2022-06-01 19:00:...|42534799|            nexsport|
|2022-06-01 19:00:...|49190631|Tìm kiếm bằng giọ...|
|2022-06-01 19:00:...|    NULL|Lương Sơn Bá Chúc...|
|2022-06-01 19:00:...|91485275|một mảnh băng t

In [12]:
ds_month6 = ds_month6.drop('datetime')

In [13]:
ds_month6 = ds_month6.groupBy('user_id','keyword').agg(count('keyword').alias('count_keyword'))

In [14]:
ds_month6.show()

+--------+--------------------+-------------+
| user_id|             keyword|count_keyword|
+--------+--------------------+-------------+
| 8460879|Đao Kiếm Thần Vực...|            1|
| 3040638|             bóng đá|            1|
|98543977|    tái sinh báo thù|            1|
| 9030455|Trực tiếp U23 Việ...|            1|
|41789116|   tiên kiếm kỳ hiệp|            1|
|91999697|        bóng đá u23 |            1|
| 8221328|           kênh vtv6|            1|
| 0789087|    giao hữu quốc tế|            1|
| 0508207|không thèm yêu đư...|            3|
|42363805|             bóng đá|            1|
|49603005|truc tiep bong đá...|            1|
|42797857|hãy gọi tôi là tổ...|            1|
|42527878|Liên Minh Công Lý...|            1|
|90282848|           5 đứa trẻ|            1|
|44179430|trực tiếp đá bóng...|            1|
|49569142|             DA BANH|            1|
|90592078|         on football|            2|
|96932783|    woori the virgin|            1|
|49195827|     u23 vief nam...|   

In [15]:
from pyspark.sql.window import Window

In [16]:
window_spec = Window.partitionBy('user_id').orderBy(desc('count_keyword'))

In [17]:
ds_month6 = ds_month6.withColumn('rank_count',dense_rank().over(window_spec))

In [18]:
ds_month6.show()

+-------+--------------------+-------------+----------+
|user_id|             keyword|count_keyword|rank_count|
+-------+--------------------+-------------+----------+
|0000402|tieu nuong tu nha...|            1|         1|
|0000402|xem phim nhan duy...|            1|         1|
|0002235|   phàm nhân tu tiên|            1|         1|
|0002235|nhanh và nguy hiểm 9|            1|         1|
|0003361|         tây hành kỷ|            1|         1|
|0003361|    tình yêu giả dối|            1|         1|
|0003361|           vuot nguc|            1|         1|
|0003361|    yêu nhầm chị dâu|            1|         1|
|0003371|                 vtv|            2|         1|
|0003371|         tây ban nha|            1|         2|
|0003371|         on football|            1|         2|
|0004287|          thuyếtminh|            2|         1|
|0004287|    người thầy y đức|            1|         2|
|0005187|          Kim Tae Ri|            1|         1|
|0005748|   lời nguyền ma lai|            2|    

In [19]:
ds_month6 = ds_month6.filter(col('user_id')!= "NULL")

In [20]:
ds_month6.show(1000,truncate=False)

+-------+-------------------------------------------------------------+-------------+----------+
|user_id|keyword                                                      |count_keyword|rank_count|
+-------+-------------------------------------------------------------+-------------+----------+
|0000402|tieu nuong tu nhatuong quan tap1                             |1            |1         |
|0000402|xem phim nhan duyen tien dinh thailan                        |1            |1         |
|000104 |HAU                                                          |1            |1         |
|000104 |bằng chứng thép                                              |1            |1         |
|0002235|nhanh và nguy hiểm 9                                         |1            |1         |
|0002235|phàm nhân tu tiên                                            |1            |1         |
|0003361|tây hành kỷ                                                  |1            |1         |
|0003361|tình yêu giả dối     

In [21]:
ds_month6.groupBy('keyword').agg(count('keyword').alias('count_keyword')).orderBy(desc('count_keyword')).show(1000,truncate=False)

+--------------------------------------------------------------------------------------------------+-------------+
|keyword                                                                                           |count_keyword|
+--------------------------------------------------------------------------------------------------+-------------+
|Liên Minh Công Lý: Phiên bản của Zack Snyder                                                      |6757         |
|thiên nga bóng đêm                                                                                |4343         |
|sao băng                                                                                          |3509         |
|nữ thanh tra tài ba                                                                               |3496         |
|bắt ma phá án                                                                                     |3388         |
|fairy tail                                                                     

In [22]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def get_category(keyword):
    completion = client.chat.completions.create(
    model="TheBloke/Mistral-7B-OpenOrca-GGUF",
    messages=[
      {"role": "system", "content": "Find the category of film for my keyword. Just give me only one category  and do not explain"},
      {"role": "user", "content": keyword}
    ],
    temperature=0.7,
  )
    return completion.choices[0].message

In [23]:
ds_month6.show()

+-------+--------------------+-------------+----------+
|user_id|             keyword|count_keyword|rank_count|
+-------+--------------------+-------------+----------+
|0000402|tieu nuong tu nha...|            1|         1|
|0000402|xem phim nhan duy...|            1|         1|
| 000104|                 HAU|            1|         1|
| 000104|     bằng chứng thép|            1|         1|
|0002235|nhanh và nguy hiểm 9|            1|         1|
|0002235|   phàm nhân tu tiên|            1|         1|
|0003361|         tây hành kỷ|            1|         1|
|0003361|    tình yêu giả dối|            1|         1|
|0003361|           vuot nguc|            1|         1|
|0003361|    yêu nhầm chị dâu|            1|         1|
|0003371|                 vtv|            2|         1|
|0003371|         tây ban nha|            1|         2|
|0003371|         on football|            1|         2|
|0004287|          thuyếtminh|            2|         1|
|0004287|    người thầy y đức|            1|    

In [24]:
get_category("thiên nga bóng đêm")

ChatCompletionMessage(content='Action & Adventure', role='assistant', function_call=None, tool_calls=None)

In [25]:
keywords_rows = ds_month6.select('keyword').collect()

In [26]:
keywords_rows

[Row(keyword='Đao Kiếm Thần Vực phần 5'),
 Row(keyword='bóng đá'),
 Row(keyword='tái sinh báo thù'),
 Row(keyword='Trực tiếp U23 Việt Nam U23 Afghanistan'),
 Row(keyword='tiên kiếm kỳ hiệp'),
 Row(keyword='bóng đá u23 '),
 Row(keyword='kênh vtv6'),
 Row(keyword='giao hữu quốc tế'),
 Row(keyword='không thèm yêu đương với sếp'),
 Row(keyword='bóng đá'),
 Row(keyword='truc tiep bong đá Việt Nam'),
 Row(keyword='hãy gọi tôi là tổng giám đốc trọn bộ'),
 Row(keyword='Liên Minh Công Lý: Phiên bản của Zack Snyder'),
 Row(keyword='5 đứa trẻ'),
 Row(keyword='trực tiếp đá bóng việt nam'),
 Row(keyword='DA BANH'),
 Row(keyword='on football'),
 Row(keyword='woori the virgin'),
 Row(keyword='     u23 vief nam _ atghanistan'),
 Row(keyword='trực tiếp việt nam giao hữu tối nay'),
 Row(keyword='việt nam vs af'),
 Row(keyword='buổi tập thứ hai của đt u23 việt nam tại uzbekistan - bản tin bóng đá việt 01/06'),
 Row(keyword='eve'),
 Row(keyword='Liên Minh Công Lý: Phiên bản của Zack Snyder'),
 Row(keyword

In [27]:
keywords_rows= ds_month6.groupBy('keyword').agg(count('keyword').alias('count_keyword')).orderBy(desc('count_keyword')).collect()

In [28]:
keywords_list = [row['keyword'] for row in keywords_rows]

In [29]:
keywords_list

['Liên Minh Công Lý: Phiên bản của Zack Snyder',
 'thiên nga bóng đêm',
 'sao băng',
 'nữ thanh tra tài ba',
 'bắt ma phá án',
 'fairy tail',
 'mộng hoa lục',
 'cô nàng trong trắng oh woo ri',
 'tôi thấy hoa vàng trên cỏ xanh',
 'siêu nhân',
 'why her?',
 'cuộc chiến thượng lưu',
 'lấy danh nghĩa người nhà',
 'cảnh đẹp ngày vui biết bao giờ',
 'bạn trai tôi là hồ ly',
 'eve',
 'vô tình nhặt được tổng tài',
 'yêu nhầm chị dâu',
 'running man',
 'yêu trong đau thương',
 'Thử Thách Thần Tượng',
 'giữa thanh xuân',
 'vẻ đẹp đích thực',
 'conan',
 'naruto',
 'lưu ly mỹ nhân sát',
 'chàng hậu',
 'em là thành trì doanh lũy của anh',
 'cuộc sống thượng lưu',
 'shooting stars',
 'hội pháp sư',
 'tại sao lại là oh soo jae?',
 'jujutsu kaisen',
 'vườn sao băng',
 'người thầy y đức',
 'yêu tinh',
 'hậu duệ mặt trời',
 'hoa của quỷ',
 'Tìm kiếm bằng giọng nói',
 'penthouse 2',
 'lưỡi gươm diệt quỷ',
 'nhất kiến khuynh tâm',
 'true beauty',
 'thanh gươm diệt quỷ: phần kỹ viện trấn',
 'nhất dạ tân 

In [30]:
# import re
# category=[]
# pattern = r"content\s*=\s*'([^']*)'"
# for i in keywords_list:
#     string = get_category(i)
#     string = str(string)
#     match = re.search(pattern, string)
#     if match:
#         category.append(match.group(1))
#     else:
#         category.append("Not Found")

In [31]:
# category

In [32]:
import pandas as pd

In [33]:
# df = pd.DataFrame(zip(keywords_category,category),columns=['keyword','category'])

In [34]:
# df.to_csv('D:\\Long Class Data\\DE\\Recent class\\Dataset\\log_search\\keyword_category.csv',index=False)

In [35]:
category_month6 = spark.read.csv('D:\\Long Class Data\\DE\\Recent class\\Final Project Product\\keyword_category_dsmonth6.csv',header=True)  

In [36]:
ds_final6 = ds_month6.join(category_month6,on='keyword',how='left')

In [37]:
ds_final6.printSchema()

root
 |-- keyword: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- count_keyword: long (nullable = false)
 |-- rank_count: integer (nullable = false)
 |-- category: string (nullable = true)



In [38]:
ds_final6.show()

+--------------------+-------+-------------+----------+------------------+
|             keyword|user_id|count_keyword|rank_count|          category|
+--------------------+-------+-------------+----------+------------------+
|tieu nuong tu nha...|0000402|            1|         1|              NULL|
|xem phim nhan duy...|0000402|            1|         1|              NULL|
|                 HAU| 000104|            1|         1|              NULL|
|     bằng chứng thép| 000104|            1|         1|       Documentary|
|nhanh và nguy hiểm 9|0002235|            1|         1|          Action\n|
|   phàm nhân tu tiên|0002235|            1|         1|            Action|
|         tây hành kỷ|0003361|            1|         1|  Action/Adventure|
|    tình yêu giả dối|0003361|            1|         1|              NULL|
|           vuot nguc|0003361|            1|         1|              NULL|
|    yêu nhầm chị dâu|0003361|            1|         1|           Romance|
|                 vtv|000

In [39]:
ds_final6= ds_final6.fillna("Not Found",subset=['category'])

In [40]:
ds_final6.show()

+--------------------+-------+-------------+----------+------------------+
|             keyword|user_id|count_keyword|rank_count|          category|
+--------------------+-------+-------------+----------+------------------+
|tieu nuong tu nha...|0000402|            1|         1|         Not Found|
|xem phim nhan duy...|0000402|            1|         1|         Not Found|
|                 HAU| 000104|            1|         1|         Not Found|
|     bằng chứng thép| 000104|            1|         1|       Documentary|
|nhanh và nguy hiểm 9|0002235|            1|         1|          Action\n|
|   phàm nhân tu tiên|0002235|            1|         1|            Action|
|         tây hành kỷ|0003361|            1|         1|  Action/Adventure|
|    tình yêu giả dối|0003361|            1|         1|         Not Found|
|           vuot nguc|0003361|            1|         1|         Not Found|
|    yêu nhầm chị dâu|0003361|            1|         1|           Romance|
|                 vtv|000

In [41]:
ds_final6 = ds_final6.filter(col('rank_count')==1)

In [42]:
ds_final6.show()

+--------------------+-------+-------------+----------+------------------+
|             keyword|user_id|count_keyword|rank_count|          category|
+--------------------+-------+-------------+----------+------------------+
|tieu nuong tu nha...|0000402|            1|         1|         Not Found|
|xem phim nhan duy...|0000402|            1|         1|         Not Found|
|                 HAU| 000104|            1|         1|         Not Found|
|     bằng chứng thép| 000104|            1|         1|       Documentary|
|nhanh và nguy hiểm 9|0002235|            1|         1|          Action\n|
|   phàm nhân tu tiên|0002235|            1|         1|            Action|
|         tây hành kỷ|0003361|            1|         1|  Action/Adventure|
|    tình yêu giả dối|0003361|            1|         1|         Not Found|
|           vuot nguc|0003361|            1|         1|         Not Found|
|    yêu nhầm chị dâu|0003361|            1|         1|           Romance|
|                 vtv|000

In [43]:
ds_final6 = ds_final6.groupby('user_id').agg(first('keyword').alias('keyword'),first('category').alias('category'))

In [44]:
ds_final6.show()

+-------+--------------------+--------------------+
|user_id|             keyword|            category|
+-------+--------------------+--------------------+
|0000402|tieu nuong tu nha...|           Not Found|
| 000104|                 HAU|           Not Found|
|0002235|nhanh và nguy hiểm 9|            Action\n|
|0003361|         tây hành kỷ|    Action/Adventure|
|0003371|                 vtv|   Action, Adventure|
|0004287|          thuyếtminh|           Not Found|
|0005187|          Kim Tae Ri|           Not Found|
|0005748|   lời nguyền ma lai|              Horror|
|0007284| hen ho chon cong so|        Comedy Drama|
|0007416|        cowboy bebop|           Not Found|
|0007511|xin goi toi la to...|           Not Found|
|0007564|        hai thế giới|           Not Found|
|0007838|      phố trong làng|               Urban|
|0008207|               CONAN|    Action/Adventure|
|0008416|               tenis|               Sport|
|0008425|lấy danh nghĩa ng...| lấy danh nghĩa n...|
|0008880|   

In [45]:
largest_original_file = None
for i in range(1,15):
    if i <10:
        largest_file = 0
        filename = "2022070" + str(i)
        for data_file in os.listdir(path + filename):
            file_size = os.path.getsize(path+filename+'\\'+data_file)
            if int(file_size) > largest_file:
                largest_filename = path + filename + '\\' + data_file
                largest_file = file_size
            else:
                continue
    else:
        filename = "202207" + str(i)
        largest_file = 0
        for data_file in os.listdir(path + filename):
            file_size = os.path.getsize(path+filename+'\\'+data_file)
            if int(file_size) > largest_file:
                largest_filename = path + filename + '\\' + data_file
                largest_file = file_size
    if largest_original_file == None:
        largest_file = largest_filename.replace("'\'","\\")
        read_newfile = spark.read.parquet(largest_file)
        largest_original_file = read_newfile
        print(f'Done {i}')
    else:
        largest_file = largest_filename.replace("'\'","\\")
        read_newfile = spark.read.parquet(largest_file)
        largest_original_file = largest_original_file.union(read_newfile)
        print(f'Done {i}')          


Done 1
Done 2
Done 3
Done 4
Done 5
Done 6
Done 7
Done 8
Done 9
Done 10
Done 11
Done 12
Done 13
Done 14


In [46]:
ds_month7 = largest_original_file

In [47]:
ds_month7.show()

+--------------------+--------------------+--------+--------------------+--------+---------+--------------------+-----------+------+--------------------+
|             eventID|            datetime| user_id|             keyword|category|proxy_isp|            platform|networkType|action|        userPlansMap|
+--------------------+--------------------+--------+--------------------+--------+---------+--------------------+-----------+------+--------------------+
|b5c61ef0-5282-442...|2022-07-01 20:25:...|42681698|                NULL|    quit|  viettel|smarttv-sony-android|        ###|search|[MAX XMAS:direct,...|
|bf4fb653-0dfc-4d8...|2022-07-01 20:25:...|98331846|    NHUNG NGUOI THUA|    quit|  viettel|smarttv-sony-android|       wifi|search|                  []|
|c2501fdb-6041-48f...|2022-07-01 20:25:...|98424407|               HAPPI|    quit|      fpt|   fplay-ottbox-2019|       wifi|search|[Kênh Trả Phí:pro...|
|f76a1565-cf65-412...|2022-07-01 20:25:...|    NULL|            hậu cung|   

In [48]:
ds_month7 = ds_month7.select('datetime','user_id','keyword')

In [49]:
ds_month7.show()

+--------------------+--------+--------------------+
|            datetime| user_id|             keyword|
+--------------------+--------+--------------------+
|2022-07-01 20:25:...|42681698|                NULL|
|2022-07-01 20:25:...|98331846|    NHUNG NGUOI THUA|
|2022-07-01 20:25:...|98424407|               HAPPI|
|2022-07-01 20:25:...|    NULL|            hậu cung|
|2022-07-01 20:25:...|06571359|vietnam next top ...|
|2022-07-01 20:28:...|47382357| tình yêu từ 0 đến 1|
|2022-07-01 20:25:...|    NULL|      tro lai toi 18|
|2022-07-01 20:25:...| 7016618|           phim thái|
|2022-07-01 20:25:...|98320714|kẻ trộm mặt trăng...|
|2022-07-01 20:25:...|40385123|              sở hữu|
|2022-07-01 20:25:...|    NULL|              bố già|
|2022-07-01 20:25:...|43952444|          fairy tail|
|2022-07-01 20:25:...|    NULL|      cô vợ bắt buộc|
|2022-07-01 20:25:...|41374324|cảnh đẹp ngày vui...|
|2022-07-01 20:26:...|93954197|boku no hero acad...|
|2022-07-01 20:26:...|06848771|      prime pro

In [50]:
ds_month7= ds_month7.dropna(subset=['user_id'])

In [51]:
ds_month7.show()

+--------------------+--------+--------------------+
|            datetime| user_id|             keyword|
+--------------------+--------+--------------------+
|2022-07-01 20:25:...|42681698|                NULL|
|2022-07-01 20:25:...|98331846|    NHUNG NGUOI THUA|
|2022-07-01 20:25:...|98424407|               HAPPI|
|2022-07-01 20:25:...|06571359|vietnam next top ...|
|2022-07-01 20:28:...|47382357| tình yêu từ 0 đến 1|
|2022-07-01 20:25:...| 7016618|           phim thái|
|2022-07-01 20:25:...|98320714|kẻ trộm mặt trăng...|
|2022-07-01 20:25:...|40385123|              sở hữu|
|2022-07-01 20:25:...|43952444|          fairy tail|
|2022-07-01 20:25:...|41374324|cảnh đẹp ngày vui...|
|2022-07-01 20:26:...|93954197|boku no hero acad...|
|2022-07-01 20:26:...|06848771|      prime provider|
|2565-07-01 20:26:...|95455337|    fairy tail movie|
|2022-07-01 20:26:...|49635591|phim 15 năm chờ đ...|
|2022-07-01 20:26:...|91154475|               v.i.p|
|2022-07-01 20:26:...|49167649|em là thành tri

In [52]:
ds_month7 = ds_month7.dropna(subset=['keyword'])

In [53]:
ds_month7.show()

+--------------------+--------+--------------------+
|            datetime| user_id|             keyword|
+--------------------+--------+--------------------+
|2022-07-01 20:25:...|98331846|    NHUNG NGUOI THUA|
|2022-07-01 20:25:...|98424407|               HAPPI|
|2022-07-01 20:25:...|06571359|vietnam next top ...|
|2022-07-01 20:28:...|47382357| tình yêu từ 0 đến 1|
|2022-07-01 20:25:...| 7016618|           phim thái|
|2022-07-01 20:25:...|98320714|kẻ trộm mặt trăng...|
|2022-07-01 20:25:...|40385123|              sở hữu|
|2022-07-01 20:25:...|43952444|          fairy tail|
|2022-07-01 20:25:...|41374324|cảnh đẹp ngày vui...|
|2022-07-01 20:26:...|93954197|boku no hero acad...|
|2022-07-01 20:26:...|06848771|      prime provider|
|2565-07-01 20:26:...|95455337|    fairy tail movie|
|2022-07-01 20:26:...|49635591|phim 15 năm chờ đ...|
|2022-07-01 20:26:...|91154475|               v.i.p|
|2022-07-01 20:26:...|49167649|em là thành trì ...|
|2022-07-01 20:26:...|90805966|            tế 

In [54]:
ds_month7 =ds_month7.drop('datetime')

In [55]:
ds_month7.show()

+--------+--------------------+
| user_id|             keyword|
+--------+--------------------+
|98331846|    NHUNG NGUOI THUA|
|98424407|               HAPPI|
|06571359|vietnam next top ...|
|47382357| tình yêu từ 0 đến 1|
| 7016618|           phim thái|
|98320714|kẻ trộm mặt trăng...|
|40385123|              sở hữu|
|43952444|          fairy tail|
|41374324|cảnh đẹp ngày vui...|
|93954197|boku no hero acad...|
|06848771|      prime provider|
|95455337|    fairy tail movie|
|49635591|phim 15 năm chờ đ...|
|91154475|               v.i.p|
|49167649|em là thành trì ...|
|90805966|            tế công |
| 4904933|   bằng chứng thép 3|
| 7018473|Tìm kiếm bằng giọ...|
|06524338|bạn trai tôi là h...|
|47758697|sự trỗi dậy của k...|
+--------+--------------------+
only showing top 20 rows



In [56]:
ds_month7 = ds_month7.groupBy('user_id','keyword').agg(count('keyword').alias('count_keyword'))

In [57]:
ds_month7.show()

+--------+--------------------+-------------+
| user_id|             keyword|count_keyword|
+--------+--------------------+-------------+
|44555675|những kẻ đào tẩu ...|            1|
|06850515|           NOOT NOOT|            3|
|43508355|      truy tìm ký ức|            1|
|49131361|           HAY NHAM |            1|
|48013076|   godzilla vs. kong|            1|
|41124571|               JADOO|            1|
|49478052| thương ngày nắng về|            1|
|94362364|     pháp y tần minh|            1|
|43225262|        lửa hận naka|            1|
|98818143|        eternal love|            1|
|97251221|    người thầy y đức|            1|
|47360012|vào ma giới rồi đ...|            1|
|41909230| cuộc chiến hậu cung|            1|
|41238725|      hoang lạc tụng|            1|
|42079221|         nhạc bolero|            1|
| 8528606|           tây du ký|            1|
| 3525924| thiên hạ lương điền|            1|
|06877281|boku no hero acad...|            1|
|06580507|     Tokyo ghoul ss4|   

In [58]:
from pyspark.sql.window import Window

In [59]:
window_spec = Window.partitionBy('user_id').orderBy(desc('count_keyword'))

In [60]:
ds_month7 = ds_month7.withColumn('rank_count',dense_rank().over(window_spec))

In [61]:
ds_month7.show()

+-------+--------------------+-------------+----------+
|user_id|             keyword|count_keyword|rank_count|
+-------+--------------------+-------------+----------+
|0000699|             thả thí|            1|         1|
|0001213|Liên Minh Công Lý...|            2|         1|
|0001465|  viet nam vs mianma|            1|         1|
|0001487|      con tim sắt đá|            1|         1|
|0001487|              the k2|            1|         1|
|0003361|         lộc đỉnh ký|            1|         1|
|0003371|hoang anh gia lai...|            1|         1|
|0003371|                 vtv|            1|         1|
|0004521|          CHEN THANH|            1|         1|
|0004675|        container 39|            2|         1|
|0004675|                 U19|            1|         2|
|0004675|                   U|            1|         2|
|0004675|            cinterne|            1|         2|
|0004999|   vụ mất tích bí ẩn|            1|         1|
|0004999|  thế giới song song|            1|    

In [62]:
ds_month7 = ds_month7.filter(col('rank_count')==1)

In [63]:
ds_month7 = ds_month7.groupBy('user_id').agg(first('keyword').alias('keyword'))

In [64]:
ds_month7.show()

+-------+--------------------+
|user_id|             keyword|
+-------+--------------------+
|0000699|             thả thí|
|0001213|Liên Minh Công Lý...|
|0001465|  viet nam vs mianma|
|0001487|      con tim sắt đá|
|0003361|         lộc đỉnh ký|
|0003371|hoang anh gia lai...|
|0004521|          CHEN THANH|
|0004675|        container 39|
|0004999|   vụ mất tích bí ẩn|
|0005187|high school king ...|
|0005748|chuyen sinh thanh...|
|0006349|            DANG XAU|
|0008207|     TIENG ANH LOP 5|
|0008226|        youth of may|
|0009683|     khu vườn bí mật|
|0010023|   phàm nhân tu tiên|
|0010311|liên minh công lý...|
|0010401|        NHU Y TRUYEN|
|0010652|              bolero|
| 001094|          UNG GIA VE|
+-------+--------------------+
only showing top 20 rows



In [65]:
list_count = ds_month7.groupBy('keyword').agg(count('keyword').alias('count_keyword')).orderBy(desc('count_keyword')).collect()

In [66]:
list_count

[Row(keyword='Liên Minh Công Lý: Phiên bản của Zack Snyder', count_keyword=4404),
 Row(keyword='thiên nga bóng đêm', count_keyword=2217),
 Row(keyword='thanh thanh tử khâm', count_keyword=2187),
 Row(keyword='con tim sắt đá', count_keyword=2016),
 Row(keyword='fairy tail', count_keyword=1780),
 Row(keyword='tôi thấy hoa vàng trên cỏ xanh', count_keyword=1774),
 Row(keyword='chàng hậu', count_keyword=1543),
 Row(keyword='nhất kiến khuynh tâm', count_keyword=1436),
 Row(keyword='kẻ trộm mặt trăng: minions', count_keyword=1347),
 Row(keyword='anna', count_keyword=1316),
 Row(keyword='siêu nhân', count_keyword=1313),
 Row(keyword='yêu tinh', count_keyword=1305),
 Row(keyword='u19', count_keyword=1066),
 Row(keyword='minh châu rực rỡ', count_keyword=1050),
 Row(keyword='conan', count_keyword=1038),
 Row(keyword='vẻ đẹp đích thực', count_keyword=1006),
 Row(keyword='sao băng', count_keyword=1002),
 Row(keyword='Thử Thách Thần Tượng', count_keyword=984),
 Row(keyword='running man', count_keyw

In [67]:
keywords_list = [row['keyword'] for row in list_count]

In [68]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

def get_category(keyword):
    completion = client.chat.completions.create(
    model="TheBloke/Mistral-7B-OpenOrca-GGUF",
    messages=[
      {"role": "system", "content": "Find the category of film for my keyword. Just give me only one category  and do not explain"},
      {"role": "user", "content": keyword}
    ],
    temperature=0.7,
  )
    return completion.choices[0].message

In [69]:
import re
category=[]
pattern = r"content\s*=\s*'([^']*)'"
for i in keywords_list:
    string = get_category(i)
    string = str(string)
    match = re.search(pattern, string)
    if match:
        category.append(match.group(1))
    else:
        category.append("Not Found")

KeyboardInterrupt: 

In [70]:
len(category)

228

In [71]:
keywords_listextract = keywords_list[:len(category)]

In [72]:
df = pd.DataFrame(zip(keywords_listextract,category),columns=['keyword','category'])

In [73]:
df

,keyword,category
0,Liên Minh Công Lý: Phiên bản của Zack Snyder,Action/Adventure
1,thiên nga bóng đêm,Horror\n
2,thanh thanh tử khâm,Action|
3,con tim sắt đá,Thriller
4,fairy tail,anime
...,...,...
223,phim hoạt hình,Anime
224,cô nàng trong trắng oh woo ri,Action and Adventure\n
225,bleach,action|
226,phim thái,action


In [74]:
df.to_csv('D:\\Long Class Data\\DE\\Recent class\\Dataset\\log_search\\keyword_category_dsmonth7.csv',index=False)

In [75]:
category_month7 = spark.read.csv('D:\\Long Class Data\\DE\\Recent class\\Final Project Product\\keyword_category_dsmonth7.csv',header=True)

In [77]:
category_month7.show()

+--------------------+--------------------+
|             keyword|            category|
+--------------------+--------------------+
|Liên Minh Công Lý...|    Action/Adventure|
|  thiên nga bóng đêm|            Horror\n|
| thanh thanh tử khâm|             Action||
|      con tim sắt đá|            Thriller|
|          fairy tail|               anime|
|tôi thấy hoa vàng...|               Drama|
|           chàng hậu|            action\n|
|nhất kiến khuynh tâm|       Action, Drama|
|kẻ trộm mặt trăng...|           Minions\n|
|                anna|              Horror|
|           siêu nhân|              Action|
|            yêu tinh|             Romance|
|                 u19|            Action\n|
|    minh châu rực rỡ|  Drama <|im_end|>\n|
|               conan|    Action/Adventure|
|    vẻ đẹp đích thực|    Action/Adventure|
|            sao băng|    Action/Adventure|
|Thử Thách Thần Tượng|Không tìm thấy kế...|
|         running man|              action|
|                 eve|          

In [78]:
ds_month7final = ds_month7.join(category_month7,on='keyword',how='left')

In [79]:
ds_month7final = ds_month7final.dropna()

In [80]:
ds_month7final.show()

+--------------------+-------+--------------------+
|             keyword|user_id|            category|
+--------------------+-------+--------------------+
|Liên Minh Công Lý...|0001213|    Action/Adventure|
|      con tim sắt đá|0001487|            Thriller|
|     khu vườn bí mật|0009683|Action, Adventure...|
|   phàm nhân tu tiên|0010023|              Action|
|              bolero|0010652|             Romance|
|                 u19|0013655|            Action\n|
|         penthouse 2|0014631|                 XXX|
|             bão cát|0016249|   horror | thriller|
|         hội pháp sư|0018156|               Drama|
|   lê hấp đường phèn|0019706|              Comedy|
|              bolero|0019920|             Romance|
|              minion|0026765|     action/comedy\n|
| hẹn hò chốn công sở|0026967|             Romance|
|     minh lan truyện|0027835|              action|
|  con ruột con riêng|0028436|     action, crime\n|
|       bắt ma phá án|0031571|              action|
|           

In [81]:
ds_final6.show()

+-------+--------------------+--------------------+
|user_id|             keyword|            category|
+-------+--------------------+--------------------+
|0000402|tieu nuong tu nha...|           Not Found|
| 000104|                 HAU|           Not Found|
|0002235|nhanh và nguy hiểm 9|            Action\n|
|0003361|         tây hành kỷ|    Action/Adventure|
|0003371|                 vtv|   Action, Adventure|
|0004287|          thuyếtminh|           Not Found|
|0005187|          Kim Tae Ri|           Not Found|
|0005748|   lời nguyền ma lai|              Horror|
|0007284| hen ho chon cong so|        Comedy Drama|
|0007416|        cowboy bebop|           Not Found|
|0007511|xin goi toi la to...|           Not Found|
|0007564|        hai thế giới|           Not Found|
|0007838|      phố trong làng|               Urban|
|0008207|               CONAN|    Action/Adventure|
|0008416|               tenis|               Sport|
|0008425|lấy danh nghĩa ng...| lấy danh nghĩa n...|
|0008880|   

In [82]:
ds_final6 = ds_final6.withColumnsRenamed({'keyword':'most_search6','category':'category6'})

In [83]:
ds_final6.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- most_search6: string (nullable = true)
 |-- category6: string (nullable = true)



In [84]:
ds_month7final = ds_month7final.withColumnsRenamed({'keyword':'most_search7','category':'category7'})

In [85]:
ds_final = ds_final6.join(ds_month7final,on='user_id',how='inner')

In [87]:
ds_final.show()

+-------+--------------------+--------------------+--------------------+--------------------+
|user_id|        most_search6|           category6|        most_search7|           category7|
+-------+--------------------+--------------------+--------------------+--------------------+
|0019706|   bên nhau trọn đời|               Drama|   lê hấp đường phèn|              Comedy|
|0019920|           thiếu nhi|               Drama|              bolero|             Romance|
|0027835|lời nói dối của h...|           Not Found|     minh lan truyện|              action|
|0028436|       bắt ma phá án| Action | Crime |...|  con ruột con riêng|     action, crime\n|
|0032305|            đám cưới|           Not Found|                anna|              Horror|
|0041173|detective k: secr...|           Not Found|                anna|              Horror|
|0042290|  thiên nga bóng đêm|  Action/Adventure\n|        võ tắc thiên|Action, Adventure...|
|0042834|   nhất dạ tân nương|            action\n|tại sao l

In [88]:
ds_final = ds_final.withColumn('Trending_Type',when(col('category6') == col('category7'),lit('Unchanged')).otherwise(lit('Changed')))

In [89]:
ds_final.show()

+-------+--------------------+--------------------+--------------------+--------------------+-------------+
|user_id|        most_search6|           category6|        most_search7|           category7|Trending_Type|
+-------+--------------------+--------------------+--------------------+--------------------+-------------+
|0019706|   bên nhau trọn đời|               Drama|   lê hấp đường phèn|              Comedy|      Changed|
|0019920|           thiếu nhi|               Drama|              bolero|             Romance|      Changed|
|0027835|lời nói dối của h...|           Not Found|     minh lan truyện|              action|      Changed|
|0028436|       bắt ma phá án| Action | Crime |...|  con ruột con riêng|     action, crime\n|      Changed|
|0032305|            đám cưới|           Not Found|                anna|              Horror|      Changed|
|0041173|detective k: secr...|           Not Found|                anna|              Horror|      Changed|
|0042290|  thiên nga bóng đê

In [91]:
ds_final = ds_final.withColumn('Previous',when(col('Trending_Type')=='Unchanged',lit("Unchanged")).otherwise(concat_ws('-',col('most_search6'),col('category7'))))

In [93]:
ds_final.show(truncate=False)

+-------+-------------------------------------+--------------------------------------------+--------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id|most_search6                         |category6                                   |most_search7              |category7                                                                                                                                                        |Trending_Type|Previous                                                                                                                                                                           |
+-------+-------------------------------------+-